<a href="https://colab.research.google.com/github/RC11-SkillsClass2022-23/KexinGao_Augustine/blob/main/2.3_Vectorisation_of_texts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pprint
import urllib.request
import nltk.corpus
import random
import os
import gensim

In [2]:
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
file_path = '/content/drive/MyDrive/texts/spy/The_Janissary_Tree.txt'
Istanbul_book = open(file_path, 'r')

In [5]:
file_content = Istanbul_book.read()

In [27]:
combined_paragraphs = []
paragraph_book_index = []

paragraphs = file_content.split('\n\n')
paragraphs_2 = [x for x in paragraphs if len(x)>80]

for p in paragraphs_2:
  combined_paragraphs.append(p)

In [28]:
stoplist = set('for a of the end to in and'.split(' '))
texts = [[word.replace('.','').replace(',','') for word in document.lower().split() if (word not in stoplist) and len(word)>2] for document in combined_paragraphs] # split the sentence to words (also filter out meaningless words)

to_delete = []
for i in range(len(texts)):
  t = texts[i]
  test = [word for word in t if word.isalpha()]
  if len(test)<20: 
    to_delete.append(i)
  else:
    texts[i]=test

for i in sorted(to_delete,reverse = True):
  del texts[i]
  del combined_paragraphs[i]

In [29]:
from collections import defaultdict
frequency = defaultdict(int)
for t in texts:
  for token in t:
    frequency[token] += 1

processed_corpus = [[token for token in t if frequency [token]>1] for t in texts]

In [30]:
print(processed_corpus[0])

['marquise', 'had', 'fished', 'out', 'paper', 'from', 'her', 'was', 'tapping', 'table', 'like', 'little', 'hammer', 'took', 'closer', 'look', 'was', 'little', 'hammer']


In [31]:
frequency['marquise']

4

In [32]:
from gensim import corpora
dictionary = corpora.Dictionary(processed_corpus)

In [33]:
print(dictionary)

Dictionary<4013 unique tokens: ['closer', 'fished', 'from', 'had', 'hammer']...>


In [34]:
pprint.pprint(dictionary.token2id)

{'abandon': 3630,
 'abandoned': 3531,
 'ability': 241,
 'able': 2043,
 'about': 484,
 'above': 772,
 'abruptly': 2761,
 'absolutely': 1040,
 'absorb': 3355,
 'abuse': 3156,
 'abused': 1872,
 'access': 3039,
 'accident': 2778,
 'accompanied': 1010,
 'accosted': 2463,
 'account': 641,
 'accounts': 2102,
 'accuse': 3092,
 'accused': 3729,
 'achieve': 928,
 'achieved': 621,
 'aching': 3581,
 'acknowledged': 3239,
 'acre': 1820,
 'across': 73,
 'act': 399,
 'acted': 3790,
 'action': 3409,
 'acts': 2997,
 'actually': 2044,
 'added': 712,
 'administration': 2556,
 'admiration': 2871,
 'admire': 1495,
 'admit': 3037,
 'admitted': 2269,
 'adopt': 1476,
 'adopted': 1161,
 'advanced': 2284,
 'advantage': 1578,
 'advertised': 1266,
 'advice': 2663,
 'affair': 1817,
 'afford': 2950,
 'afraid': 1139,
 'african': 1617,
 'after': 1140,
 'afterward': 1862,
 'again': 1083,
 'against': 294,
 'age': 3248,
 'agha': 1827,
 'ago': 74,
 'agony': 1754,
 'agreed': 3410,
 'ahead': 2568,
 'ahmet': 2902,
 'aimee':

In [35]:
new_doc = 'zeki zeki zeynep'
new_vec = dictionary.doc2bow(new_doc.lower().split())

In [36]:
print(new_vec)

[]


In [37]:
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]

In [38]:
from gensim import models
tfidf = models.TfidfModel(bow_corpus)

In [40]:
words = 'The mysterious city of Istanbul'.lower().split()
print(tfidf[dictionary.doc2bow(words)])

[(25, 0.4178309651768902), (169, 0.35618428177058137), (703, 0.8357930616838856)]


In [41]:
from gensim.similarities import Similarity
from gensim.test.utils import common_corpus, common_dictionary, get_tmpfile

In [42]:
index_tmpfile = get_tmpfile('index')
index = Similarity(index_tmpfile, bow_corpus, num_features = len(dictionary))

In [43]:
query_document = 'Hagia Sopia was once a church, but was turned to a mosque.'.lower().split()
query_bow = dictionary.doc2bow(query_document)

sims = index[query_bow]

In [44]:
for document_number, score in sorted(enumerate(sims), key = lambda x:x[1], reverse = True):
  print(document_number, score)

834 0.48653364
971 0.4762897
831 0.47513112
1078 0.42600647
56 0.4170288
951 0.41646338
64 0.4152274
366 0.40016335
361 0.4
952 0.3944053
713 0.39086798
265 0.39013717
762 0.38235956
87 0.37712365
620 0.3742276
959 0.36440545
333 0.36128742
341 0.36004117
692 0.36004117
669 0.35015053
997 0.34978145
682 0.34815535
973 0.34752402
12 0.34299716
830 0.34299716
206 0.3414317
58 0.34035164
310 0.3380617
865 0.33686078
940 0.33671755
7 0.3350126
1068 0.33333334
108 0.3307593
48 0.32998317
518 0.32824397
355 0.32732686
154 0.32075018
863 0.320508
665 0.31647512
615 0.3162278
660 0.31497037
661 0.31497037
853 0.31497037
542 0.30949223
230 0.30905753
1064 0.3063817
935 0.30588764
314 0.30499715
444 0.30499715
884 0.30429032
576 0.30002344
227 0.29981267
476 0.2993422
639 0.29875273
858 0.29875273
626 0.2981424
1010 0.2981424
799 0.2963336
1046 0.29462782
645 0.2909572
170 0.28171808
839 0.28005603
521 0.27801922
687 0.2777778
911 0.2777778
222 0.27399832
393 0.27399832
827 0.27216554
343 0.2712